# Introduction to Apache Spark

### Import Reuired Libraries

In [7]:
from pyspark.sql import SparkSession

### Create SparkSession and SparkContext

In [8]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

### Demonstrate Spark's Lazy behaviour

In [10]:
file_rdd = sc.textFile("gs://dataproc-staging-279317/data/lang123.txt")

In [11]:
rdd1 = file_rdd.map(lambda x : x.split(" "))

In [12]:
rdd2 = rdd1.map(lambda x : (x, 1))

In [13]:
rdd2.count()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: gs://dataproc-staging-279317/data/lang123.txt
	at org.apache.hadoop.mapred.LocatedFileStatusFetcher.getFileStatuses(LocatedFileStatusFetcher.java:155)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:322)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:204)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:273)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:269)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:273)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:269)
	at org.apache.spark

### Create a RDD from a collection.

In [14]:
num = [1,2,3,4,5]
num_rdd = sc.parallelize(num)
print("Type of num_rdd : ", type(num_rdd))
print("Elements in num_rdd : ", num_rdd.collect())

Type of num_rdd :  <class 'pyspark.rdd.RDD'>
Elements in num_rdd :  [1, 2, 3, 4, 5]


## Transformations
* As we know, Transformations are lazy in nature and they will not be executed until an Action is executed on top of them. 
* Let’s try to understand various available Transformations.

### map
* This will map your input to some output based on the function specified in the map function.

In [15]:
double_rdd = num_rdd.map(lambda x : x * 2)
double_rdd.collect()

[2, 4, 6, 8, 10]

### filter
* To filter the data based on a certain condition. Let’s try to find the even numbers from num_rdd.

In [16]:
even_rdd = num_rdd.filter(lambda x : x % 2 == 0)
even_rdd.collect()

[2, 4]

### flatMap
* This function is very similar to map, but can return multiple elements for each input in the given RDD.

In [17]:
map_rdd = num_rdd.map(lambda x : list(range(1,x)))
map_rdd.collect()

[[], [1], [1, 2], [1, 2, 3], [1, 2, 3, 4]]

In [18]:
flat_rdd = num_rdd.flatMap(lambda x : list(range(1,x)))
flat_rdd.collect()

[1, 1, 2, 1, 2, 3, 1, 2, 3, 4]

### distinct
* This will return distinct elements from an RDD.

In [19]:
rdd1 = sc.parallelize([10, 11, 10, 11, 12, 11])
dist_rdd = rdd1.distinct()
dist_rdd.collect()

[10, 12, 11]

### reduceByKey
* This function reduces the key values pairs based on the keys and a given function inside the reduceByKey. Here’s an example.

In [20]:
pairs = [ ("a", 5), ("b", 7), ("c", 2), ("a", 3), ("b", 1), ("c", 4)]
pair_rdd = sc.parallelize(pairs)
pair_rdd.collect()

# <a, (5, 3)>
# <b, (7, 1)>
# <c, (2, 4)>

[('a', 5), ('b', 7), ('c', 2), ('a', 3), ('b', 1), ('c', 4)]

In [22]:
output = pair_rdd.reduceByKey(lambda x, y : x * y)

result = output.collect()
print(*result, sep='\n')

('b', 7)
('c', 8)
('a', 15)


### groupByKey
* This function is another ByKey function which can operate on a (key, value) pair RDD but this will only group the values based on the keys. In other words, this will only perform the first step of reduceByKey.

In [23]:
grp_out = pair_rdd.groupByKey()
grp_out.mapValues(list).collect()

[('b', [7, 1]), ('c', [2, 4]), ('a', [5, 3])]

### sortByKey
* This function will perform the sorting on a (key, value) pair RDD based on the keys. By default, sorting will be done in ascending order.

In [24]:
pairs = [ ("a", 5), ("d", 7), ("c", 2), ("b", 3)]
raw_rdd = sc.parallelize(pairs)
print(*raw_rdd.collect(), sep='\n')
# Note: for sorting in descending order pass “ascending=False”.

('a', 5)
('d', 7)
('c', 2)
('b', 3)


In [25]:
sortkey_rdd = raw_rdd.sortByKey()

result = sortkey_rdd.collect()
print(*result,sep='\n')

('a', 5)
('b', 3)
('c', 2)
('d', 7)


### sortBy
* sortBy is a more generalized function for sorting.


In [26]:
# Create RDD.
pairs = [ ("a", 5, 10), ("d", 7, 12), ("c", 2, 11), ("b", 3, 9)]
raw_rdd = sc.parallelize(pairs)

print(*raw_rdd.collect(), sep='\n')

('a', 5, 10)
('d', 7, 12)
('c', 2, 11)
('b', 3, 9)


In [27]:
# Let’s try to do the sorting based on the 3rd element of the tuple.
sort_out = raw_rdd.sortBy(lambda x : x[2])

result = sort_out.collect()
print(*result, sep='\n')

('b', 3, 9)
('a', 5, 10)
('c', 2, 11)
('d', 7, 12)


## Actions
* Actions are operations on RDDs which execute immediately. While Transformations return another RDD, Actions return language native data structures.

### count
* This will count the number of elements in the given RDD.

In [28]:
num = sc.parallelize([1,2,3,4,2])
num.count()

5

### first
* This will return the first element from given RDD.

In [29]:
num.first()

1

### Collect
* This will return all the elements for the given RDD.


In [30]:
num.collect()

[1, 2, 3, 4, 2]

 Note: We should not use the collect operation while working with large datasets. Because it will return all the data which is distributed across the different workers of your cluster to a driver. All the data will travel across the network from worker to driver and also the driver would need to hold all the data. This will hamper the performance of your application.

### Take
* This will return the number of elements specified.


In [32]:
num.take(2)

[1, 2]

### Reference
* https://medium.com/expedia-group-tech/start-your-journey-with-apache-spark-part-2-682891efda4b
* https://spark.apache.org/docs/latest/rdd-programming-guide.html?source=post_page3575b20ee088#actions

## Next Session : 23rd June 2020
#### Registration Link : https://www.meetup.com/Artificial-Intelligence-UK/events/271081272/


### Focus :  Spark SQL API / DataFrame
* Introduction to Spark DataFrame
* Data Frame Operations
* Read Data from HIVE table
* Save Data to HIVE Table
* Read/Write data with CSV
* Read/Write data with Parquet

## Thank You